# Welcome to Lexicographic Genetic Optimization

In this tutorial, we will have a look at genetic algorithms with a lexicographic genetic fitness function and their application to module composition optimization.

For more information, have a look at our [paper](https://arxiv.org/abs/2209.06758) "Optimizing Modular Robot Composition: A Lexicographic Genetic Algorithm Approach" and the corresponding [website](https://lexicographic-ga.cps.cit.tum.de/).

The implementation of the Genetic Algorithm is based on `pygad`. You can find the documentation of the library [here](https://pygad.readthedocs.io/en/latest/).

In [ ]:
import pygad
import timor

## What is a lexicographic fitness?
Simply said, a lexicographic fitness function is a **hierarchical** structure that contains multiple scalar values that are arranged in a strict order.
Based on these values, we can define a lexicographic **ordering** that is used during the selection phase of a genetic algorithm.
Timor comes with its own implementation of a lexicographic value:

In [ ]:
from timor.utilities.dtypes import Lexicographic

In [ ]:
attributes = {  # Assume we computed the following lexicographic values for four different solution candidates
    1: (0.5, 0.8, 0.1),
    2: (0.2, 0.8, 0.1),
    3: (0.5, 0.4, 0.1),
    4: (0.5, 0.8, 0.4),
}

# We can manually create a lexicographic value from these attributes
fitness_one = Lexicographic(*attributes[1])
fitness_two = Lexicographic(*attributes[2])
fitness_three = Lexicographic(*attributes[3])
fitness_four = Lexicographic(*attributes[4])
all_fitness = (fitness_one, fitness_two, fitness_three, fitness_four)

# It is now possible to compare them:
print("The first fitness is {} than the second fitness".format("larger" if fitness_one > fitness_two else "smaller"))

Even better, numpy can also work with the Lexicographic data type.

In [ ]:
import numpy as np
order = np.argsort(all_fitness)
print("The order of fitness values, in ascending order, is:", ', '.join(str(i + 1) for i in order))
print("The values of the highest fitness are:", np.max(all_fitness).values)

One more nice thing about the lexicographic type: It doesn't matter which elements you assign to it, as long as they support the < and > operation:

In [ ]:
print("a > b", "a" > "b")
print("C > D", "C" > "D")
print("Lexicographic Comparison 1:", Lexicographic("a", "a") > Lexicographic("a", "b"))
print("Lexicographic Comparison 2:", Lexicographic("D", "b") > Lexicographic("C", "a"))

In [ ]:
# You can even nest them:
nested_one = Lexicographic(fitness_one, fitness_two)
nested_two = Lexicographic(fitness_two, fitness_one)
print(nested_one > nested_two)

# Genetic Algorithms with Timor
In timor, we use GAs to optimize module compositions. In order to do so, we first need some modules.

In [ ]:
from timor import ModuleAssembly, ModulesDB
from timor.configuration_search.GA import GA

In [ ]:
db = ModulesDB.from_name('modrob-gen2')  # This loads the module library we used in "Optimizing Modular Robot Composition: A Lexicographic Genetic Algorithm Approach"

We want to visualize our modules and the optimization results. In timor, we use the meshcat visualizer for that purpose. For more details, refer to the other tutorials:

In [ ]:
from timor.utilities.visualization import MeshcatVisualizer, clear_visualizer
viz = MeshcatVisualizer()
viz.initViewer()
db.debug_visualization(viz)  # Let's have a look at the modules we just loaded

## Setting up the optimization
Next to the modules, we can also define custom hyperparameters, and of course, we need to define a **fitness function**:

The fitness function needs to fulfill the following criteria:
- It takes three input arguments: 1) A ModuleAssembly, 2) a pygad.GA instance, 3) an integer indicating the index of a solution candidate of the population
- It doesn't need to use these arguments, but you probably want it to use at least the assembly.
- It returns a single value, the **fitness value**. This value should either be of datatype `float` or `Lexicographic`. It is important that this return type should always be the same.

In [ ]:
our_hyperparameters = {
    'population_size': 10,
    'num_generations': 50,
    'num_genes': 6,
    'save_solutions_dir': None
}

In [ ]:
def fitness_scalar(assembly: ModuleAssembly, ga_instance: pygad.GA, index: int) -> float:
    """We start with a very simple fitness function that returns the negative mass of the assembly"""
    return -assembly.mass

In [ ]:
ga = GA(db, our_hyperparameters)  # We set up an instance of the optimization algorithm by providing the modules we want to use and the hyperparameters. If we do not provide custom hyperparameters, timor defaults will be used

## Starting the optimization
After setting up the optimization instance and the fitness function, we can start with the optimization.

In this case, we would expect that the algorithm finds assemblies with a small weight, as our fitness criterion is given by the mass of the robot. Let's see:

In [ ]:
ga_optimizer = ga.optimize(fitness_function=fitness_scalar, save_best_solutions=False)

In [ ]:
print("The best robot in the initial population had a mass of:", -ga_optimizer.best_solutions_fitness[0], "kg")
print("The best robot had a mass of:", -ga_optimizer.best_solution()[1], "kg")
print("The best robot had", len([module for module in ga_optimizer.best_solution()[0] if module != 0]), "modules")

Apparently, the GA was able to optimize the robot, leading to a lightweight robot. It did so by deleting as many modules as possible from the robot. Next, Let's define a lexicographic fitness function and see if the mass can be optimized without this "hack":

In [ ]:
def fitness_lexicographic(assembly: ModuleAssembly, ga_instance: pygad.GA, index: int) -> Lexicographic:
    """
    This fitness function returns a lexicographic value, where the
    
    - first value indicates the number of modules in the robot, and
    - the second value the minus of the total mass
    """
    return Lexicographic(len(assembly.module_instances), -assembly.mass)

In [ ]:
ga_optimizer = ga.optimize(fitness_function=fitness_lexicographic, save_best_solutions=False)  # we can use the same GA instance, but with the new fitness function

In [ ]:
print("The best robot in the initial population had {} modules and a mass of {:.3f}kg.".format(ga_optimizer.best_solutions_fitness[0].values[0], -ga_optimizer.best_solutions_fitness[0].values[1]))
print("The best robot had {} modules and a mass of {:.3f}kg.".format(ga_optimizer.best_solution()[1].values[0], -ga_optimizer.best_solution()[1].values[1]))

## How to proceed from here on?

This tutorial just transmits some basics. For further reference, please refer to the documentation of the GA module (here)[https://timor-python.readthedocs.io/en/latest/autoapi/timor/configuration_search/GA/index.html]. There you can learn how to:

- Change default hyperparameters
- Track the progress of the optimization with weights and biases (wandb)
- How to cache intermediate results to save runtime
- How to include different callbacks in the optimization to achieve various logging and visualization features
- How to define custom mutation, crossover, or selection algorithms